# SimEX Loop
We will be asked to create a "coach" loop that organize the training of a controller, which will do the following 

The following is pseudo-code which shall not necessarily represent the real method signatures, but shall indicate the flow of logic, presenting where which information is used and consumed.  
```bash
init trainingData 

counter = 0 

newTrainingData = {} 

Do 

counter++ 

trainingData = trainingData + newTrainingData 

controller.doTraining(trainingData) 

PathtoCSV= runSimEx(controller) //creates new csv outputs 
 - Seems to only be the simulator? providing new function in the simulator as parameter (retrained model?) 

newTrainingData = checkPerformance(PathToReference, PathToCSV).  

// method provide by Martin, reading csv output from simex, but is a simple loop testing for fixed areas if their performance is higher or lower than the reference (computing the value of the provide polynom, identifiyable via the intervals in the csv) and if the currentPerfomance is worse than reference add datapoints for new training data (approximate at least reference values as targets)).   

while(newTrainingData !={} AND counter<MaxIteration)
```
methods in need to be provided by HR side, but they need our help to give them a structure to work in 

Note, with this loop you can theoretically implement zero learning (from untrained to hero automatically 

Note 2: this loops enable the interesting question, when do we stop learning, even though results are not perfect, e.g. when we have to acknowledge given available actions, we cannot solve the problem (as for the moment, what is "learned" is the action-selection, while primitive actions are pre-defined. I.e. when the set of actions is not sufficient no learning can be successful, an alternative when to stopp before I overfit... * all of this are interesting research questions (for later work) 

In [ ]:
from notebooks.modifier_controller import ModifierController

print("hell")

In [ ]:
import subprocess
import pandas as pd
import os
os.environ['INSTANCE_NAME'] = 'LOOP_script'
from global_settings import SimexSettings
script_dir = os.path.abspath('')
results_dir = os.path.join(script_dir, f'{SimexSettings.results_dir}')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

from DLASIUT_find_best_scenarios import automatic_performance

scripts = ['sumo_novsl_run.py', 'sumo_vsl_run.py']
print("start")
files = []
for script in scripts:
    cmd = ['python3', script]
    lines = subprocess.run(cmd,stdout=subprocess.PIPE).stdout.splitlines()
    print(f" LINES: {lines}")
    baseline_file = None
    for line in lines:
        output = "{}".format(line.rstrip().decode("utf-8"))
        if 'simex_output' in output:
            files.append(output)
    
     # collect name of the csv file
    if len(files) == 2:
        df_baseline = pd.read_csv(files[1])
        df_control = pd.read_csv(files[0])
        # Main Function receives np.arrays
        dataset_baseline = df_baseline.to_numpy()
        dataset_control = df_control.to_numpy()

        # Run martin script on the csv files
        _incremnet_step_for_x = 10
        _max_order_of_polynom = 9
        _tolerance_in_diffrence = 12
        results = automatic_performance(dataset_baseline, dataset_control, incremnet_step_for_x=_incremnet_step_for_x,
                                        max_order_of_polynom=_max_order_of_polynom,
                                        tolerance_in_diffrence=_tolerance_in_diffrence)
        print(f"Results: {results}")      

In [ ]:
import pandas as pd

import subprocess
import pandas as pd
import os
os.environ['INSTANCE_NAME'] = 'LOOP_script'
from global_settings import SimexSettings
script_dir = os.path.abspath('')
results_dir = os.path.join(script_dir, f'{SimexSettings.results_dir}')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

from DLASIUT_find_best_scenarios import automatic_performance
files = ['/home/amy/tmp/repos/SimEx/notebooks/results_dir_NOVSL_script-20240902-113858/simex_output-NOVSL_script-20240902-113858.csv','/home/amy/tmp/repos/SimEx/notebooks/results_dir_VSL_script-20240902-120605/simex_output-VSL_script-20240902-120605.csv']
print(f"Files {files}")
df_baseline = pd.read_csv(files[1])
df_control = pd.read_csv(files[0])
# Main Function receives np.arrays
dataset_baseline = df_baseline.to_numpy()
dataset_control = df_control.to_numpy()

# Run martin script on the csv files
_incremnet_step_for_x = 10
_max_order_of_polynom = 9
_tolerance_in_diffrence = 12
results = automatic_performance(dataset_baseline, dataset_control, incremnet_step_for_x=_incremnet_step_for_x,
                                        max_order_of_polynom=_max_order_of_polynom,
                                        tolerance_in_diffrence=_tolerance_in_diffrence)
print(f"Results: {results}") 

In [ ]:

def run_simex(simulator_function, instance_name):    
    # IMPORT LIBRARIES
    import os
    import argparse
    import numpy as np
    # Set instance name
    # os.environ['INSTANCE_NAME'] = instance_name
    
    from global_settings import SimexSettings, Mds, timestamp, Fs
    SimexSettings.instance_name = instance_name
    print(f"Instance name {SimexSettings.instance_name}")
    resultdir = f"results_dir_{instance_name}-{timestamp}"
    # Create directory for the results
    script_dir = os.path.abspath('')
    results_dir = os.path.join(script_dir, resultdir)
    
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    
    from components_configuration import components
    from validator_controller import ValidatorController
    from modifier_controller import ModifierController
    from simulator_controller import SimulatorController
    from logger_utils import Logger
    
    import pickle
    import datetime
    
    def save_object(obj, filename):
        with open(filename, 'wb') as outp:  # Overwrites any existing file.
            pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    
    logger = Logger()
    validator_controller_vsl = ValidatorController()
    # logger = Logger()
    logger_main_arguments = {}
    is_main_func = True
    # Initialize interval list for the first iteration
    
    
    intervals_list = [[Mds.domain_min_interval, Mds.domain_max_interval]]
    # Timestamp for the validator pickle file
    count = 0
    
    while is_main_func:
        # Calls Modifier Controller
        # NOTE: intervals_list type is set to np.int64 due to: https://github.com/numpy/numpy/issues/8433 on windows
        mod_outcome = ModifierController.control(intervals_list=intervals_list, selected_modifier=components['modifierA'],
                                                 do_plot=SimexSettings.do_plot)
        mod_x_list = mod_outcome[0]
        checked_intervals = mod_outcome[1]
        print("MAIN mod outcome", mod_outcome)
    
        # breaks loop if iterations end by granularity reached
        if not mod_x_list:  # FALSE IF ['modifier_data_point'] < mdv['modifier_incremental_unit']:
            logger_main_arguments['log_contex'] = 'overall MAIN stats'
            logger_main_arguments['main_status'] = 'no generated points'
            logger_main_arguments['remaining_unfit_intervals'] = checked_intervals
            logger.log_main(logger_main_arguments)
            break
    
        # Calls Simulator
        mod_x, sim_y_list = SimulatorController.simulate_parallel(mod_x_list, selected_simulator=simulator_function)
        print(f"MODX {mod_x} and sim_y_list {sim_y_list}")
        assert len(mod_x) == len(sim_y_list)
    
        print("MAIN modx", mod_x)
    
        # Calls Validator controller
        intervals_list = validator_controller_vsl.validate(mod_x_list=np.array(mod_x), sim_y_list=np.array(sim_y_list),
                                                           selected_validator=components['validator'],
                                                           global_interval=[Mds.domain_min_interval,
                                                                            Mds.domain_max_interval])
        print("MAIN interval list from VAL:", intervals_list)
        # Loop number ( Loop-1,Loop-2..etc)
        count += 1
        save_object(validator_controller_vsl, os.path.join(results_dir, f"vc_vsl_loop-{count}-{timestamp}.pkl"))
    
        # Updates interval_list to new range output from validator controller
        # No more unfit intervals -> write MAIN log
        if not intervals_list:
            is_main_func = False
            logger_main_arguments['log_contex'] = 'overall MAIN stats'
            logger_main_arguments['main_status'] = 'no unfit intervals'
            logger.log_main(logger_main_arguments)
    
    # MAIN cycle completed/interrupted -> write OVERALL statistics
    logger_main_arguments['log_contex'] = 'Overall Stats'
    logger_main_arguments['main_status'] = 'end cycle'
    logger.log_main(logger_main_arguments)
    
    # Save data for the last plot located in logger object
    save_object(logger.all_fit_intervals_data,os.path.join(results_dir,f"logger-vsl_script-fitted_intervals-{timestamp}.pkl"))
    # If not empty
    if logger.remaining_unfit_intervals:
        save_object(logger.remaining_unfit_intervals,os.path.join(results_dir,f"logger-vsl_script-unfitted_intervals-{timestamp}.pkl"))
    # print(f"Logger object saved with timestamp {timestamp}")
    file = f"{Fs.csv_filename}-{timestamp}.csv"
    file_path = os.path.join(results_dir,file)
    print(f"{file_path}")
    return file_path

In [1]:

import pandas as pd

from DLASIUT_find_best_scenarios import automatic_performance
from simulator import Simulator

from run_simex import run_simex

# Run baseline NOVSL case


# Rewrite function to accept different types of input data and different simulator function
base_file = run_simex(simulator_function=Simulator.sumo_simulator_novsl, instance_name='NOVSL_base_case')
# collect name of the csv file
if base_file:
    df_baseline = pd.read_csv(base_file)
    # Main Function receives np.arrays
    dataset_baseline = df_baseline.to_numpy()
    counter = 0
    MaxIteration = 10
    newTrainingData = []
    while True:
        counter +=1
        # trainingData = trainingData + newTrainingData 

        # Insert training of the controller
        # controller.doTraining(trainingData)
        controller = Simulator.sumo_simulator_vsl
        # Run simex for the retrained controller replace components['']
        control_file = run_simex(simulator_function=controller, instance_name='VSL')
        df_control = pd.read_csv(control_file)
        dataset_control = df_control.to_numpy()
        # Run martin script on the csv files
        _incremnet_step_for_x = 10
        _max_order_of_polynom = 9
        _tolerance_in_diffrence=12
        bad_regions = automatic_performance(dataset_baseline,dataset_control,incremnet_step_for_x=_incremnet_step_for_x,max_order_of_polynom=_max_order_of_polynom,tolerance_in_diffrence=_tolerance_in_diffrence)
        print(bad_regions)
        if bad_regions:
            from modifier_controller import ModifierController
            from modifier import Modifier
            newTrainingData = ModifierController.control(intervals_list=bad_regions, selected_modifier=Modifier.modifierA, do_plot=SimexSettings.do_plot)
            print(newTrainingData)
        if not newTrainingData and counter<MaxIteration:
            break



Instance name NOVSL_base_case


FileNotFoundError: [Errno 2] No such file or directory: 'results_dir_-20240908-140047/LOG---20240908-140047.txt'